In [ ]:
# ✅ Install dependencies (if not already)
!pip install -q --upgrade scikit-learn==1.1.3 imbalanced-learn==0.10.1

# ✅ Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, accuracy_score, cohen_kappa_score,
                             precision_score, recall_score, f1_score,
                             roc_auc_score, matthews_corrcoef)
from imblearn.metrics import geometric_mean_score, specificity_score

# ✅ Pretrained models
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50V2, InceptionV3, InceptionResNetV2,
    DenseNet121, DenseNet169, DenseNet201, Xception, EfficientNetB0,
    MobileNetV3Large
)

In [ ]:
# ✅ Dataset paths
violence_folder = '/kaggle/input/hdataset/Hdataset/Fight'
nonviolence_folder = '/kaggle/input/hdataset/Hdataset/Nofight'


In [ ]:

# ✅ Load image function
def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        try:
            img = load_img(path, target_size=(224, 224))
            img = img_to_array(img) / 255.0
            images.append(img)
            labels.append(label)
        except:
            print(f"Error loading {path}")
    return np.array(images), np.array(labels)

In [ ]:
# ✅ Build model
def build_transfer_learning_model(base_model_fn, input_shape=(224, 224, 3), lr=0.001):
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# ✅ Evaluation function

def evaluate(model, X_test, y_test, model_name):
    y_probs = model.predict(X_test)
    y_pred = (y_probs > 0.5).astype(int).flatten()
    print(f"\n📊 Classification Report for {model_name}:")
    print(classification_report(y_test, y_pred))

    acc = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    auroc = roc_auc_score(y_test, y_probs, average='weighted')
    mcc = matthews_corrcoef(y_test, y_pred)
    spec = specificity_score(y_test, y_pred, average='weighted')
    gmean = geometric_mean_score(y_test, y_pred, average='weighted')
    type_i = 1 - gmean
    type_ii = 1 - spec

    return [model_name, acc, kappa, prec, rec, f1, auroc, mcc, spec, gmean, type_i, type_ii]

In [ ]:
# ✅ Load and prepare data
v_images, v_labels = load_images_from_folder(violence_folder, 1)
nv_images, nv_labels = load_images_from_folder(nonviolence_folder, 0)
X = np.concatenate((v_images, nv_images), axis=0)
y = np.concatenate((v_labels, nv_labels), axis=0)
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"✅ Dataset: {len(y)} samples")

In [ ]:
 #✅ Models to train
models = [
    ("VGG16", VGG16),
    ("VGG19", VGG19),
    ("MobileNetV3Large", MobileNetV3Large),
    ("ResNet50V2", ResNet50V2),
    ("InceptionResNetV2", InceptionResNetV2),
    ("DenseNet121", DenseNet121),
    ("DenseNet169", DenseNet169),
    ("DenseNet201", DenseNet201),
    ("InceptionV3", InceptionV3),
    ("EfficientNetB0", EfficientNetB0),
    ("Xception", Xception)
]

In [ ]:
# ✅ Train, evaluate and collect results
results = []
for name, model_fn in models:
    print(f"\n🚀 Training {name}...")
    model = build_transfer_learning_model(model_fn)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32, verbose=1)
    results.append(evaluate(model, X_test, y_test, name))


In [ ]:
# ✅ Save all results to CSV
results_df = pd.DataFrame(results, columns=[
    "Model", "Accuracy", "Cohen-Kappa", "Precision", "Recall", "F1-Score",
    "AUROC", "MCC", "Specificity", "G-Mean", "Type I Error", "Type II Error"])
results_df.to_csv("/kaggle/working/all_model_results.csv", index=False)
print("\n✅ Results saved to /kaggle/working/all_model_results.csv")
display(results_df)